In [1]:
import sys
assert sys.version_info >= (3, 5)

import sklearn
import numpy as np
import pandas as pd

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns

loans = pd.read_csv("Loan_Default.csv")
print("done")

done


In [2]:
clean_loans =  loans.drop(['year'],axis=1)

In [3]:
clean_loans = clean_loans.dropna(axis=0, subset=['loan_purpose'])

clean_loans = clean_loans.dropna(axis=0, subset=['term','age','submission_of_application','Neg_ammortization'])

In [4]:
clean_loans['loan_limit'] = clean_loans[['loan_limit']].fillna('cf')


In [5]:
clean_loans['approv_in_adv'] = clean_loans[['approv_in_adv']].fillna('nopre')

clean_loans['income'] = clean_loans['income'].fillna(clean_loans['income'].median())


In [6]:
conditions = [clean_loans['dtir1'].isnull() & clean_loans['Status'].eq(1), clean_loans['dtir1'].isnull() & clean_loans['Status'].eq(0),
              clean_loans['dtir1'].isnull() == False]

choices = [clean_loans['dtir1'][clean_loans['Status']==1].median(),clean_loans['dtir1'][clean_loans['Status']==0].median(), clean_loans['dtir1']]

clean_loans['dtir1'] = np.select(conditions, choices, default=0)


In [7]:
conditions = [clean_loans['property_value'].isnull() & clean_loans['Status'].eq(1), clean_loans['property_value'].isnull() & clean_loans['Status'].eq(0),
              clean_loans['property_value'].isnull() == False]

choices = [clean_loans['property_value'][clean_loans['Status']==1].median(),clean_loans['property_value'][clean_loans['Status']==0].median(), clean_loans['property_value']]

clean_loans['property_value'] = np.select(conditions, choices, default=0)


In [8]:
conditions = [clean_loans['LTV'].isnull() & clean_loans['Status'].eq(1), clean_loans['LTV'].isnull() & clean_loans['Status'].eq(0),
              clean_loans['LTV'].isnull() == False]

choices = [clean_loans['LTV'][clean_loans['Status']==1].median(),clean_loans['LTV'][clean_loans['Status']==0].median(), clean_loans['LTV']]

clean_loans['LTV'] = np.select(conditions, choices, default=0)


In [9]:
conditions = [clean_loans['Upfront_charges'].isnull() & clean_loans['Status'].eq(1), clean_loans['Upfront_charges'].isnull() & clean_loans['Status'].eq(0),
              clean_loans['Upfront_charges'].isnull() == False]

choices = [clean_loans['Upfront_charges'][clean_loans['Status']==0].median(),clean_loans['Upfront_charges'][clean_loans['Status']==0].median(), clean_loans['Upfront_charges']]

clean_loans['Upfront_charges'] = np.select(conditions, choices, default=0)


In [10]:
conditions = [clean_loans['rate_of_interest'].isnull() & clean_loans['Status'].eq(1), clean_loans['rate_of_interest'].isnull() == False]

choices = [clean_loans['rate_of_interest'][clean_loans['Status']==0].median(), clean_loans['rate_of_interest']]

clean_loans['rate_of_interest'] = np.select(conditions, choices)


In [11]:
conditions = [clean_loans['Interest_rate_spread'].isnull() & clean_loans['Status'].eq(1), clean_loans['Interest_rate_spread'].isnull() == False]

choices = [clean_loans['Interest_rate_spread'][clean_loans['Status']==0].median(),clean_loans['Interest_rate_spread']]

clean_loans['Interest_rate_spread'] = np.select(conditions, choices, default=0)

In [12]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_enc = LabelEncoder()

In [13]:
df = clean_loans.drop(["loan_limit", "Gender", "approv_in_adv", "loan_type", "loan_purpose", "Credit_Worthiness", "open_credit", 
                                           "business_or_commercial", "Neg_ammortization", "interest_only", "lump_sum_payment", "construction_type", 
                                           "occupancy_type", "Secured_by", "total_units", "credit_type", "co-applicant_credit_type", "age", "submission_of_application", 
                                           "Region", "Security_Type"], axis = 1)

In [14]:
df["loan_limit_enc"] = label_enc.fit_transform(clean_loans["loan_limit"])
df["Gender_enc"] = label_enc.fit_transform(clean_loans["Gender"])
df["approv_in_adv_enc"] = label_enc.fit_transform(clean_loans["approv_in_adv"])
df["loan_type_enc"] = label_enc.fit_transform(clean_loans["loan_type"])
df["loan_purpose_enc"] = label_enc.fit_transform(clean_loans["loan_purpose"])
df["Credit_Worthiness_enc"] = label_enc.fit_transform(clean_loans["Credit_Worthiness"])
df["open_credit_enc"] = label_enc.fit_transform(clean_loans["open_credit"])
df["business_or_commercial_enc"] = label_enc.fit_transform(clean_loans["business_or_commercial"])
df["Neg_ammortization_enc"] = label_enc.fit_transform(clean_loans["Neg_ammortization"])
df["interest_only_enc"] = label_enc.fit_transform(clean_loans["interest_only"])
df["lump_sum_payment_enc"] = label_enc.fit_transform(clean_loans["lump_sum_payment"])
df["construction_type_enc"] = label_enc.fit_transform(clean_loans["construction_type"])
df["occupancy_type_enc"] = label_enc.fit_transform(clean_loans["occupancy_type"])
df["Secured_by_enc"] = label_enc.fit_transform(clean_loans["Secured_by"])
df["total_units_enc"] = label_enc.fit_transform(clean_loans["total_units"])
df["credit_type_enc"] = label_enc.fit_transform(clean_loans["credit_type"])
df["co-applicant_credit_type_enc"] = label_enc.fit_transform(clean_loans["co-applicant_credit_type"])
df["age_enc"] = label_enc.fit_transform(clean_loans["age"])
df["submission_of_application_enc"] = label_enc.fit_transform(clean_loans["submission_of_application"])
df["Region_enc"] = label_enc.fit_transform(clean_loans["Region"])
df["Security_Type_enc"] = label_enc.fit_transform(clean_loans["Security_Type"])

In [15]:
loans.isna().sum()
loans = df.dropna()

In [16]:
loans["Status"].value_counts()

0    111817
1     36365
Name: Status, dtype: int64

In [17]:
approved_loans = loans[loans["Status"] == 1]
rejected_loans = loans[loans["Status"] == 0]

In [18]:
approved_loans.shape

(36365, 33)

In [19]:
rejected_loans.shape

(111817, 33)

In [20]:
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix
from sklearn.feature_selection import RFE, RFECV, SelectKBest, chi2, f_classif, SelectPercentile

In [21]:
x = loans.drop("Status", axis = 1)
y = loans["Status"]

x0 = rejected_loans.drop("Status", axis = 1)
x1 = approved_loans.drop("Status", axis = 1)

y0 = rejected_loans["Status"]
y1 = approved_loans["Status"]

In [22]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
x0_train, x0_test, y0_train, y0_test = train_test_split(x0, y0, random_state=0)
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, random_state=0)


KNN Classifier

In [23]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn0 = KNeighborsClassifier(n_neighbors=1)
knn1 = KNeighborsClassifier(n_neighbors=1)

knn.fit(x_train, y_train)
knn0.fit(x0_train, y0_train)
knn1.fit(x1_train, y1_train)

y_pred = knn.predict(x_test)
y0_pred = knn0.predict(x0_test)
y1_pred = knn1.predict(x1_test)

In [24]:
print("Complete")

Complete


In [25]:
x

,ID,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,property_value,income,Credit_Score,LTV,...,construction_type_enc,occupancy_type_enc,Secured_by_enc,total_units_enc,credit_type_enc,co-applicant_credit_type_enc,age_enc,submission_of_application_enc,Region_enc,Security_Type_enc
0,24890,116500,3.990,0.3902,2600.00,360.0,118000.0,1740.0,758,98.728814,...,1,1,0,0,3,0,0,1,3,1
1,24891,206500,3.990,0.3902,2600.00,360.0,348000.0,4980.0,552,79.336735,...,1,1,0,0,2,1,3,1,0,1
2,24892,406500,4.560,0.2000,595.00,360.0,508000.0,9480.0,834,80.019685,...,1,1,0,0,3,0,1,1,3,1
3,24893,456500,4.250,0.6810,2600.00,360.0,658000.0,11880.0,587,69.376900,...,1,1,0,0,3,0,2,0,0,1
4,24894,696500,4.000,0.3042,0.00,360.0,758000.0,10440.0,602,91.886544,...,1,1,0,0,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148665,173555,436500,3.125,0.2571,9960.00,180.0,608000.0,7860.0,659,71.792763,...,1,1,0,0,0,1,3,1,3,1
148666,173556,586500,5.190,0.8544,0.00,360.0,788000.0,7140.0,569,74.428934,...,1,0,0,3,0,0,0,0,3,1
148667,173557,446500,3.125,0.0816,1226.64,180.0,728000.0,6900.0,702,61.332418,...,1,1,0,0,0,1,2,0,0,1
148668,173558,196500,3.500,0.5824,4323.33,180.0,278000.0,7140.0,737,70.683453,...,1,1,0,0,3,1,3,1,0,1


In [26]:
y

0         1
1         1
2         0
3         0
4         0
         ..
148665    0
148666    0
148667    0
148668    0
148669    0
Name: Status, Length: 148182, dtype: int64

In [27]:
x0

,ID,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,property_value,income,Credit_Score,LTV,...,construction_type_enc,occupancy_type_enc,Secured_by_enc,total_units_enc,credit_type_enc,co-applicant_credit_type_enc,age_enc,submission_of_application_enc,Region_enc,Security_Type_enc
2,24892,406500,4.560,0.2000,595.00,360.0,508000.0,9480.0,834,80.019685,...,1,1,0,0,3,0,1,1,3,1
3,24893,456500,4.250,0.6810,2600.00,360.0,658000.0,11880.0,587,69.376900,...,1,1,0,0,3,0,2,0,0,1
4,24894,696500,4.000,0.3042,0.00,360.0,758000.0,10440.0,602,91.886544,...,1,1,0,0,1,1,0,0,0,1
5,24895,706500,3.990,0.1523,370.00,360.0,1008000.0,10080.0,864,70.089286,...,1,1,0,0,3,1,1,0,0,1
6,24896,346500,4.500,0.9998,5120.00,360.0,438000.0,5040.0,860,79.109589,...,1,1,0,0,3,1,3,1,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148665,173555,436500,3.125,0.2571,9960.00,180.0,608000.0,7860.0,659,71.792763,...,1,1,0,0,0,1,3,1,3,1
148666,173556,586500,5.190,0.8544,0.00,360.0,788000.0,7140.0,569,74.428934,...,1,0,0,3,0,0,0,0,3,1
148667,173557,446500,3.125,0.0816,1226.64,180.0,728000.0,6900.0,702,61.332418,...,1,1,0,0,0,1,2,0,0,1
148668,173558,196500,3.500,0.5824,4323.33,180.0,278000.0,7140.0,737,70.683453,...,1,1,0,0,3,1,3,1,0,1


In [28]:
x1

,ID,loan_amount,rate_of_interest,Interest_rate_spread,Upfront_charges,term,property_value,income,Credit_Score,LTV,...,construction_type_enc,occupancy_type_enc,Secured_by_enc,total_units_enc,credit_type_enc,co-applicant_credit_type_enc,age_enc,submission_of_application_enc,Region_enc,Security_Type_enc
0,24890,116500,3.99,0.3902,2600.0,360.0,118000.0,1740.0,758,98.728814,...,1,1,0,0,3,0,0,1,3,1
1,24891,206500,3.99,0.3902,2600.0,360.0,348000.0,4980.0,552,79.336735,...,1,1,0,0,2,1,3,1,0,1
10,24900,136500,3.99,0.3902,2600.0,300.0,168000.0,4020.0,723,81.250000,...,1,1,0,0,3,0,3,1,0,1
12,24902,206500,3.99,0.3902,2600.0,360.0,258000.0,3780.0,884,80.038760,...,1,1,0,0,1,1,4,1,0,1
15,24905,76500,3.99,0.3902,2600.0,360.0,138000.0,2220.0,685,55.434783,...,1,1,0,0,3,0,2,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148651,173541,446500,3.99,0.3902,2600.0,360.0,508000.0,12300.0,897,87.893701,...,1,1,0,0,3,1,2,1,0,1
148652,173542,96500,3.99,0.3902,2600.0,180.0,348000.0,5460.0,608,79.336735,...,1,0,0,0,2,1,3,1,0,1
148658,173548,386500,3.99,0.3902,2600.0,360.0,348000.0,4680.0,669,79.336735,...,1,1,0,0,2,1,0,1,3,1
148661,173551,346500,3.99,0.3902,2600.0,360.0,358000.0,5760.0,585,96.787710,...,1,1,0,0,3,0,0,1,3,1


In [29]:
y0

2         0
3         0
4         0
5         0
6         0
         ..
148665    0
148666    0
148667    0
148668    0
148669    0
Name: Status, Length: 111817, dtype: int64

In [30]:
y1

0         1
1         1
10        1
12        1
15        1
         ..
148651    1
148652    1
148658    1
148661    1
148663    1
Name: Status, Length: 36365, dtype: int64

In [31]:
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0)
x0_train, x0_test, y0_train, y0_test = train_test_split(x0, y0, random_state=0)
x1_train, x1_test, y1_train, y1_test = train_test_split(x1, y1, random_state=0)


print("done")

done


In [32]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier(n_neighbors=1)
knn0 = KNeighborsClassifier(n_neighbors=1)
knn1 = KNeighborsClassifier(n_neighbors=1)

knn.fit(x_train, y_train)
knn0.fit(x0_train, y0_train)
knn1.fit(x1_train, y1_train)

y_pred = knn.predict(x_test)
y0_pred = knn0.predict(x0_test)
y1_pred = knn1.predict(x1_test)

print("done")

done


In [33]:
print("complete")

complete


In [34]:
# Test predicted data
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

accuracy_0 = accuracy_score(y0_test, y0_pred)
f1_0 = f1_score(y0_test, y0_pred)

accuracy_1 = accuracy_score(y1_test, y1_pred)
f1_1 = f1_score(y1_test, y1_pred)

print("Accuracy:", accuracy)
print("F1 Score: ", f1)
print("\n")

print("Accuracy for 0:", accuracy_0)
print("F1 Score for 0: ", f1_0)
print("\n")

print("Accuracy for 1:", accuracy_1)
print("F1 Score for 1: ", f1_1)
print("\n")

print( "\n\nHeatMap:\n", pd.crosstab(pd.Series(y_pred, name = "Predicted"), pd.Series(y_test, name = "Actual")))
print("\n")

print( "\n\nHeatMap for rejected:\n", pd.crosstab(pd.Series(y0_pred, name = "Predicted"), pd.Series(y0_test, name = "Actual")))
print("\n")

print( "\n\nHeatMap for approved:\n", pd.crosstab(pd.Series(y1_pred, name = "Predicted"), pd.Series(y1_test, name = "Actual")))

Accuracy: 0.7700966366139395
F1 Score:  0.5477138760554405


Accuracy for 0: 1.0
F1 Score for 0:  0.0


Accuracy for 1: 1.0
F1 Score for 1:  1.0




HeatMap:
 Actual        0     1
Predicted            
0          5091  1717
1          1807   571




HeatMap for rejected:
 Actual        0
Predicted      
0          5294




HeatMap for approved:
 Actual       1
Predicted     
1          549


/Users/ndaya/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1580: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, "true nor predicted", "F-score is", len(true_sum))


In [35]:
k_best_features = SelectKBest(f_classif, k = 5).fit(x_train, y_train)
k_best_features_df = pd.DataFrame({"Features" : list(x_train.columns), "F-Value" : k_best_features.scores_})
k_best_features_df.sort_values(by = "F-Value", ascending = False)


,Features,F-Value
21,lump_sum_payment_enc,4034.530403
19,Neg_ammortization_enc,2630.627749
10,dtir1,2349.974365
27,co-applicant_credit_type_enc,2322.155872
29,submission_of_application_enc,1686.235529
6,property_value,1512.648288
26,credit_type_enc,1494.958861
4,Upfront_charges,1011.417721
18,business_or_commercial_enc,931.253195
7,income,452.924643


In [36]:
# Select top 2 features
k_best_features_2_train = x_train.drop(["submission_of_application_enc", "business_or_commercial_enc", "Upfront_charges", "LTV", "loan_type_enc", "dtir1",
                                  "co-applicant_credit_type_enc", "rate_of_interest", "Interest_rate_spread", "income", "property_value", "age_enc", "loan_limit_enc", 
                                  "Gender_enc", "loan_amount", "Region_enc", "Secured_by_enc", "construction_type_enc", "Security_Type_enc", "approv_in_adv_enc", 
                                  "occupancy_type_enc", "total_units_enc","Credit_Worthiness_enc", "open_credit_enc", "term", "credit_type_enc", "loan_purpose_enc", 
                                  "Credit_Score", "interest_only_enc", "ID"], axis = 1)

k_best_features_2_test = x_test.drop(["submission_of_application_enc", "business_or_commercial_enc", "Upfront_charges", "LTV", "loan_type_enc", "dtir1",
                                  "co-applicant_credit_type_enc", "rate_of_interest", "Interest_rate_spread", "income", "property_value", "age_enc", "loan_limit_enc", 
                                  "Gender_enc", "loan_amount", "Region_enc", "Secured_by_enc", "construction_type_enc", "Security_Type_enc", "approv_in_adv_enc", 
                                  "occupancy_type_enc", "total_units_enc","Credit_Worthiness_enc", "open_credit_enc", "term", "credit_type_enc", "loan_purpose_enc", 
                                  "Credit_Score", "interest_only_enc", "ID"], axis = 1)

################################################################################
# Select top 5 features
k_best_features_5_train = x_train.drop(["LTV", "loan_type_enc", "dtir1", "co-applicant_credit_type_enc", "rate_of_interest", "Interest_rate_spread", "income", "property_value", 
                                  "age_enc", "loan_limit_enc", "Gender_enc", "loan_amount", "Region_enc", "Secured_by_enc", "construction_type_enc", "Security_Type_enc", 
                                  "approv_in_adv_enc", "occupancy_type_enc", "total_units_enc","Credit_Worthiness_enc", "open_credit_enc", "term", "credit_type_enc", 
                                  "loan_purpose_enc", "Credit_Score", "interest_only_enc", "ID"], axis = 1)


k_best_features_5_test = x_test.drop(["LTV", "loan_type_enc", "dtir1", "co-applicant_credit_type_enc", "rate_of_interest", "Interest_rate_spread", "income", "property_value", 
                                  "age_enc", "loan_limit_enc", "Gender_enc", "loan_amount", "Region_enc", "Secured_by_enc", "construction_type_enc", "Security_Type_enc", 
                                  "approv_in_adv_enc", "occupancy_type_enc", "total_units_enc","Credit_Worthiness_enc", "open_credit_enc", "term", "credit_type_enc", 
                                  "loan_purpose_enc", "Credit_Score", "interest_only_enc", "ID"], axis = 1)


################################################################################
# Select top 10 features
k_best_features_10_train = x_train.drop(["Interest_rate_spread", "income", "property_value", "age_enc", "loan_limit_enc", "Gender_enc", "loan_amount", "Region_enc", "Secured_by_enc", 
                                  "construction_type_enc", "Security_Type_enc", "approv_in_adv_enc", "occupancy_type_enc", "total_units_enc","Credit_Worthiness_enc", 
                                  "open_credit_enc", "term", "credit_type_enc", "loan_purpose_enc", "Credit_Score", "interest_only_enc", "ID"], axis = 1)

k_best_features_10_test = x_test.drop(["Interest_rate_spread", "income", "property_value", "age_enc", "loan_limit_enc", "Gender_enc", "loan_amount", "Region_enc", "Secured_by_enc", 
                                  "construction_type_enc", "Security_Type_enc", "approv_in_adv_enc", "occupancy_type_enc", "total_units_enc","Credit_Worthiness_enc", 
                                  "open_credit_enc", "term", "credit_type_enc", "loan_purpose_enc", "Credit_Score", "interest_only_enc", "ID"], axis = 1)


print("done")

done


In [37]:
from sklearn.neighbors import KNeighborsClassifier
knn2 = KNeighborsClassifier(n_neighbors=1)
knn2.fit(k_best_features_2_train, y_train)
k_best_2_features_pred = knn2.predict(k_best_features_2_test)
print("done")

done


In [40]:
# Test predicted data
accuracy = accuracy_score(y_test, k_best_5_features_pred)
f1 = f1_score(y_test, k_best_5_features_pred)

print("Accuracy:", accuracy)
print("F1 Score: ", f1)
print("\nAccuracy increased from ~75% to 97% - this is a huge improvement form before")
print("F1 Score went up imensly to 0.93 which is very very good")

print( "\n\nHeatMap:\n", pd.crosstab(pd.Series(k_best_5_features_pred, name = "Predicted"),pd.Series(y_test, name = "Actual")))


NameError: name 'k_best_5_features_pred' is not defined

In [41]:
k_best_5_features_pred

NameError: name 'k_best_5_features_pred' is not defined

In [ ]:
# Test predicted data
accuracy = accuracy_score(y_test, k_best_2_features_pred)
f1 = f1_score(y_test, k_best_2_features_pred)

print("Accuracy:", accuracy)
print("F1 Score: ", f1)
print("\nAccuracy increased from ~75% to 84%")
print("F1 Score went down however")

print( "\n\nHeatMap:\n", pd.crosstab(pd.Series(k_best_2_features_pred, name = "Predicted"), pd.Series(y_test, name = "Actual")))

print("\nHeatmap also shows less false postive and false negatives")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn5 = KNeighborsClassifier(n_neighbors=1)
knn5.fit(k_best_features_5_train, y_train)
k_best_5_features_pred = knn5.predict(k_best_features_5_test)
print("done")

In [ ]:
# Test predicted data
accuracy = accuracy_score(y_test, k_best_5_features_pred)
f1 = f1_score(y_test, k_best_5_features_pred)

print("Accuracy:", accuracy)
print("F1 Score: ", f1)
print("\nAccuracy increased from ~75% to 97% - this is a huge improvement form before")
print("F1 Score went up imensly to 0.93 which is very very good")

print( "\n\nHeatMap:\n", pd.crosstab(pd.Series(k_best_5_features_pred, name = "Predicted"),pd.Series(y_test, name = "Actual")))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
knn10 = KNeighborsClassifier(n_neighbors=1)
knn10.fit(k_best_features_10_train, y_train)
k_best_10_features_pred = knn10.predict(k_best_features_10_test)
print("done")

In [ ]:
# Test predicted data
accuracy = accuracy_score(y_test, k_best_10_features_pred)
f1 = f1_score(y_test, k_best_10_features_pred)

print("Accuracy:", accuracy)
print("F1 Score: ", f1)
print("\nAccuracy falls very slightly from 5 features (0.9769) when using 10 features (0.9725)")
print("F1 Score also decreases slightly from 5 features (0.9336) when using 10 features (0.9167) ")

print( "\n\nHeatMap:\n", pd.crosstab(pd.Series(k_best_10_features_pred, name = "Predicted"), pd.Series(y_test, name = "Actual")))


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV

rfecv = RFECV(RandomForestClassifier(), scoring = "accuracy")
rfecv.fit_transform(x, y)
# rfecv.fit_transform(x_train, y_train)

print("done")

In [ ]:
x

In [ ]:
y

In [ ]:
print("xy")

In [ ]:
x_train

In [ ]:
rfecv = RFECV(RandomForestClassifier(), scoring = "accuracy")
rfecv.fit_transform(x_train, y_train)
print("done after xtyt")

In [ ]:
rfecv.fit_transform(x, y)
print("done after xy")

In [ ]:
rfecv.score(x_test,y_test)

In [ ]:
rfecv.get_feature_names_out

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import KFold, StratifiedKFold, cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix, precision_score, recall_score, precision_recall_curve, roc_curve, roc_auc_score, RocCurveDisplay

rf_model = RandomForestClassifier()

In [ ]:
rf_model.fit(x_train, y_train)

In [ ]:
rf_model.score(x_train,y_train)

In [ ]:
rf_model.score(x_test,y_test)

In [ ]:
rf_pred = rf_model.predict(x_test)
rf_accuracy = accuracy_score(y_test, rf_pred)
rf_f1 = f1_score(y_test, rf_pred)
rf_precision = precision_score(y_test, rf_pred)
rf_recall = recall_score(y_test, rf_pred)
rf_cm = confusion_matrix(y_test, rf_pred)

print("Accuracy:", rf_accuracy)
print("F1 Score: ", rf_f1)
print("Precision Score: ", rf_precision)
print("Recall Score: ", rf_recall)

print("\nConfusion Matrix: \n", rf_cm)

print("\nTP = 100% \nFP = 0%")

In [ ]:
rf_roc = RocCurveDisplay.from_estimator(rf_model, x_test, y_test)
plt.show()

In [ ]:
knn_model = KNeighborsClassifier(n_neighbors=1)
knn_model.fit(x_train, y_train)
knn_pred = knn_model.predict(x_test)

In [ ]:
knn_model.score(x_train, y_train)

In [ ]:
knn_model.score(x_test, y_test)

In [ ]:
knn_pred = knn_model.predict(x_test)
knn_accuracy = accuracy_score(y_test, knn_pred)
knn_f1 = f1_score(y_test, knn_pred)
knn_precision = precision_score(y_test, knn_pred)
knn_recall = recall_score(y_test, knn_pred)
knn_cm = confusion_matrix(y_test, knn_pred)

print("Accuracy:", knn_accuracy)
print("F1 Score: ", knn_f1)
print("Precision Score: ", knn_precision)
print("Recall Score: ", knn_recall)

print("\nConfusion Matrix: \n", knn_cm)

In [ ]:
axis = plt.gca()
knn_roc = RocCurveDisplay.from_estimator(knn_model, x_test, y_test, ax=axis, alpha=0.8)
rf_roc.plot(ax=axis, alpha=0.8)
plt.show()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
dt_model = DecisionTreeClassifier()
dt_model.fit(x_train, y_train)
dt_pred = dt_model.predict(x_test)


In [ ]:
dt_model.score(x_train, y_train)

In [ ]:
dt_model.score(x_test, y_test)

In [ ]:
dt_pred = dt_model.predict(x_test)
dt_accuracy = accuracy_score(y_test, dt_pred)
dt_f1 = f1_score(y_test, dt_pred)
dt_precision = precision_score(y_test, dt_pred)
dt_recall = recall_score(y_test, dt_pred)
dt_cm = confusion_matrix(y_test, dt_pred)

print("Accuracy:", dt_accuracy)
print("F1 Score: ", dt_f1)
print("Precision Score: ", dt_precision)
print("Recall Score: ", dt_recall)

print("\nConfusion Matrix: \n", dt_cm)

In [ ]:
axis = plt.gca()
rf_roc.plot(ax=axis, alpha=0.9)
knn_roc.plot(ax=axis, alpha=0.9)
dt_roc = RocCurveDisplay.from_estimator(dt_model, x_test, y_test, ax=axis, alpha=0.9)
plt.figure(figsize=(20,20))
plt.show()

In [ ]:
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB(alpha=0.0, class_prior=[0.4, 0.6])
model.fit(x, y)

In [ ]:
mnb_model = MultinomialNB(alpha=0.0, class_prior=[0.4, 0.6])
mnb_model.fit(x_train, y_train)

In [ ]:
mnb_model.fit(k_best_features_5_train, y_train)

In [ ]:
mnb_model.fit(k_best_features_2_train, y_train)

In [ ]:
mnb_model.fit(k_best_features_10_train, y_train)

In [ ]:
mnb_model = MultinomialNB(alpha=1.0, class_prior=[0.4, 0.6])
mnb_model.fit(k_best_features_10_train, y_train)

In [ ]:
mnb_predict = mnb_model.predict(k_best_features_10_test)

In [ ]:
mnb_model.score(k_best_features_10_train, y_train)

In [ ]:
mnb_accuracy = accuracy_score(y_test, mnb_predict)

In [ ]:
print("Accuracy:", mnb_accuracy)

In [ ]:
mnb_confusion = confusion_matrix(y_test, mnb_predict)

In [ ]:
print("Confusion:", mnb_confusion)

In [ ]:
print(mnb_confusion[0,0])

In [ ]:
print(mnb_confusion[1,1])

In [ ]:
# TP-[0,0] FN-[0,1] 
# FP-[1,0] TN-[1,1]
#Sensitivity = TP/(TP+FN)
#Specificity = TN/(TN+FP)
#Precision precision=TP/(TP+FP)
#Recall recall =TP/(TP+FN)
#F-measure =( 2 × recall × precision) / (recall+precision)
#

In [ ]:
mnb_sensitivity = mnb_confusion[0,0]/(mnb_confusion[0,0]+mnb_confusion[0,1])
print(mnb_sensitivity)

In [ ]:
mnb_specificity = mnb_confusion[1,1]/(mnb_confusion[1,1]+mnb_confusion[1,0])
print(mnb_specificity)

In [ ]:
mnb_roc_score = roc_auc_score(y_test, mnb_predict)
print(mnb_roc_score)

In [ ]:
mnb_recall = mnb_confusion[0,0]/(mnb_confusion[0,0]+mnb_confusion[0,1])
print(mnb_recall)

In [ ]:
mnb_recall_fn = recall_score(y_test, mnb_predict, average=None)
print(mnb_recall_fn)

In [ ]:
#Precision precision=TP/(TP+FP)


In [ ]:
print X

In [ ]:
print x

In [ ]:
x

In [ ]:
X